In [2]:
import pandas as pd

weather_covariate = pd.read_parquet("weather_covariate.parquet", engine="pyarrow")
weather_covariate

,dt,dt_iso,timezone,city_name,lat,lon,temp,visibility,dew_point,feels_like,...,wind_gust,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,1230768000,2009-01-01 00:00:00 +0000 UTC,-18000,Alphabet City,40.724968,-73.979102,268.04,10000.0,259.50,261.04,...,19.00,NaN,NaN,NaN,NaN,20,801,Clouds,few clouds,02n
1,1230771600,2009-01-01 01:00:00 +0000 UTC,-18000,Alphabet City,40.724968,-73.979102,266.44,10000.0,257.29,259.44,...,17.00,NaN,NaN,NaN,NaN,20,801,Clouds,few clouds,02n
2,1230775200,2009-01-01 02:00:00 +0000 UTC,-18000,Alphabet City,40.724968,-73.979102,266.44,10000.0,257.29,259.44,...,17.50,NaN,NaN,NaN,NaN,75,803,Clouds,broken clouds,04n
3,1230778800,2009-01-01 03:00:00 +0000 UTC,-18000,Alphabet City,40.724968,-73.979102,266.44,10000.0,258.23,259.44,...,14.40,NaN,NaN,NaN,NaN,0,800,Clear,sky is clear,01n
4,1230782400,2009-01-01 04:00:00 +0000 UTC,-18000,Alphabet City,40.724968,-73.979102,265.44,10000.0,258.15,258.44,...,16.50,NaN,NaN,NaN,NaN,0,800,Clear,sky is clear,01n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10607598,1704049200,2023-12-31 19:00:00 +0000 UTC,-18000,Yorkville West,40.777410,-73.950599,278.88,10000.0,270.41,275.34,...,8.75,NaN,NaN,NaN,NaN,75,803,Clouds,broken clouds,04d
10607599,1704052800,2023-12-31 20:00:00 +0000 UTC,-18000,Yorkville West,40.777410,-73.950599,279.01,10000.0,270.52,275.74,...,NaN,NaN,NaN,NaN,NaN,100,804,Clouds,overcast clouds,04d
10607600,1704056400,2023-12-31 21:00:00 +0000 UTC,-18000,Yorkville West,40.777410,-73.950599,279.01,10000.0,270.52,275.74,...,NaN,NaN,NaN,NaN,NaN,100,804,Clouds,overcast clouds,04d
10607601,1704060000,2023-12-31 22:00:00 +0000 UTC,-18000,Yorkville West,40.777410,-73.950599,278.95,10000.0,270.47,275.20,...,NaN,NaN,NaN,NaN,NaN,100,804,Clouds,overcast clouds,04n


In [4]:
zone_to_loc = pd.read_csv("zone_to_loc.csv")
zone_to_loc

,PULocationID,zone
0,4,Alphabet City
1,12,Battery Park
2,13,Battery Park City
3,24,Bloomingdale
4,41,Central Harlem
...,...,...
62,246,West Chelsea/Hudson Yards
63,249,West Village
64,261,World Trade Center
65,262,Yorkville East


In [9]:
print(list(weather_covariate['dt_iso'][:10]))

['2009-01-01 00:00:00 +0000 UTC', '2009-01-01 01:00:00 +0000 UTC', '2009-01-01 02:00:00 +0000 UTC', '2009-01-01 03:00:00 +0000 UTC', '2009-01-01 04:00:00 +0000 UTC', '2009-01-01 05:00:00 +0000 UTC', '2009-01-01 06:00:00 +0000 UTC', '2009-01-01 07:00:00 +0000 UTC', '2009-01-01 08:00:00 +0000 UTC', '2009-01-01 09:00:00 +0000 UTC']


In [11]:
weather_covariate['datetime'] = pd.to_datetime(weather_covariate['dt_iso'], format='%Y-%m-%d %H:%M:%S %z UTC')
print(list(weather_covariate['datetime'][:10]))

[Timestamp('2009-01-01 00:00:00+0000', tz='UTC'), Timestamp('2009-01-01 01:00:00+0000', tz='UTC'), Timestamp('2009-01-01 02:00:00+0000', tz='UTC'), Timestamp('2009-01-01 03:00:00+0000', tz='UTC'), Timestamp('2009-01-01 04:00:00+0000', tz='UTC'), Timestamp('2009-01-01 05:00:00+0000', tz='UTC'), Timestamp('2009-01-01 06:00:00+0000', tz='UTC'), Timestamp('2009-01-01 07:00:00+0000', tz='UTC'), Timestamp('2009-01-01 08:00:00+0000', tz='UTC'), Timestamp('2009-01-01 09:00:00+0000', tz='UTC')]


In [18]:
start_date = pd.to_datetime('2021-01-01 23:00:00').tz_localize('UTC')
end_date = pd.to_datetime('2023-10-31 23:00:00').tz_localize('UTC')

weather_covariate_time_filtered = weather_covariate[
    (weather_covariate['datetime'] >= start_date) & (weather_covariate['datetime'] <= end_date)
]
weather_covariate_time_filtered

,dt,dt_iso,timezone,city_name,lat,lon,temp,visibility,dew_point,feels_like,...,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon,datetime
112237,1609542000,2021-01-01 23:00:00 +0000 UTC,-18000,Alphabet City,40.724968,-73.979102,276.66,10000.0,273.54,272.83,...,0.57,NaN,NaN,NaN,100,500,Rain,light rain,10n,2021-01-01 23:00:00+00:00
112238,1609545600,2021-01-02 00:00:00 +0000 UTC,-18000,Alphabet City,40.724968,-73.979102,276.58,10000.0,273.97,271.57,...,0.43,NaN,NaN,NaN,100,500,Rain,light rain,10n,2021-01-02 00:00:00+00:00
112239,1609549200,2021-01-02 01:00:00 +0000 UTC,-18000,Alphabet City,40.724968,-73.979102,276.54,10000.0,274.74,271.16,...,1.98,NaN,NaN,NaN,100,501,Rain,moderate rain,10n,2021-01-02 01:00:00+00:00
112240,1609552800,2021-01-02 02:00:00 +0000 UTC,-18000,Alphabet City,40.724968,-73.979102,276.65,9656.0,275.32,271.66,...,2.03,NaN,NaN,NaN,100,501,Rain,moderate rain,10n,2021-01-02 02:00:00+00:00
112241,1609552800,2021-01-02 02:00:00 +0000 UTC,-18000,Alphabet City,40.724968,-73.979102,276.65,9656.0,275.32,271.66,...,2.03,NaN,NaN,NaN,100,701,Mist,mist,50n,2021-01-02 02:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10606007,1698778800,2023-10-31 19:00:00 +0000 UTC,-14400,Yorkville West,40.777410,-73.950599,283.69,10000.0,273.15,282.05,...,NaN,NaN,NaN,NaN,100,804,Clouds,overcast clouds,04d,2023-10-31 19:00:00+00:00
10606008,1698782400,2023-10-31 20:00:00 +0000 UTC,-14400,Yorkville West,40.777410,-73.950599,283.62,10000.0,273.37,282.00,...,NaN,NaN,NaN,NaN,100,804,Clouds,overcast clouds,04d,2023-10-31 20:00:00+00:00
10606009,1698786000,2023-10-31 21:00:00 +0000 UTC,-14400,Yorkville West,40.777410,-73.950599,283.26,10000.0,273.32,281.63,...,NaN,NaN,NaN,NaN,100,804,Clouds,overcast clouds,04d,2023-10-31 21:00:00+00:00
10606010,1698789600,2023-10-31 22:00:00 +0000 UTC,-14400,Yorkville West,40.777410,-73.950599,282.89,10000.0,273.51,281.02,...,NaN,NaN,NaN,NaN,100,804,Clouds,overcast clouds,04n,2023-10-31 22:00:00+00:00


In [19]:
for c in weather_covariate_time_filtered.columns:
    print(c)

dt
dt_iso
timezone
city_name
lat
lon
temp
visibility
dew_point
feels_like
temp_min
temp_max
pressure
sea_level
grnd_level
humidity
wind_speed
wind_deg
wind_gust
rain_1h
rain_3h
snow_1h
snow_3h
clouds_all
weather_id
weather_main
weather_description
weather_icon
datetime


In [20]:
useful_features = ['city_name', 'temp', 'humidity', 'wind_speed', "wind_gust", "rain_1h", "rain_3h", "snow_1h", "snow_3h", "clouds_all", "weather_main", "weather_description", "datetime"]
weather_covariate_time_features_filtered = weather_covariate_time_filtered[useful_features]
weather_covariate_time_features_filtered

,city_name,temp,humidity,wind_speed,wind_gust,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_main,weather_description,datetime
112237,Alphabet City,276.66,80,4.60,0.0,0.57,NaN,NaN,NaN,100,Rain,light rain,2021-01-01 23:00:00+00:00
112238,Alphabet City,276.58,83,7.20,0.0,0.43,NaN,NaN,NaN,100,Rain,light rain,2021-01-02 00:00:00+00:00
112239,Alphabet City,276.54,88,8.20,0.0,1.98,NaN,NaN,NaN,100,Rain,moderate rain,2021-01-02 01:00:00+00:00
112240,Alphabet City,276.65,91,7.20,0.0,2.03,NaN,NaN,NaN,100,Rain,moderate rain,2021-01-02 02:00:00+00:00
112241,Alphabet City,276.65,91,7.20,0.0,2.03,NaN,NaN,NaN,100,Mist,mist,2021-01-02 02:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10606007,Yorkville West,283.69,48,4.12,NaN,NaN,NaN,NaN,NaN,100,Clouds,overcast clouds,2023-10-31 19:00:00+00:00
10606008,Yorkville West,283.62,49,3.60,NaN,NaN,NaN,NaN,NaN,100,Clouds,overcast clouds,2023-10-31 20:00:00+00:00
10606009,Yorkville West,283.26,50,3.09,NaN,NaN,NaN,NaN,NaN,100,Clouds,overcast clouds,2023-10-31 21:00:00+00:00
10606010,Yorkville West,282.89,52,3.60,NaN,NaN,NaN,NaN,NaN,100,Clouds,overcast clouds,2023-10-31 22:00:00+00:00


In [24]:
weather_covariate_time_features_filtered = weather_covariate_time_features_filtered.fillna(0)
weather_covariate_time_features_filtered

,city_name,temp,humidity,wind_speed,wind_gust,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_main,weather_description,datetime
112237,Alphabet City,276.66,80,4.60,0.0,0.57,0.0,0.0,0.0,100,Rain,light rain,2021-01-01 23:00:00+00:00
112238,Alphabet City,276.58,83,7.20,0.0,0.43,0.0,0.0,0.0,100,Rain,light rain,2021-01-02 00:00:00+00:00
112239,Alphabet City,276.54,88,8.20,0.0,1.98,0.0,0.0,0.0,100,Rain,moderate rain,2021-01-02 01:00:00+00:00
112240,Alphabet City,276.65,91,7.20,0.0,2.03,0.0,0.0,0.0,100,Rain,moderate rain,2021-01-02 02:00:00+00:00
112241,Alphabet City,276.65,91,7.20,0.0,2.03,0.0,0.0,0.0,100,Mist,mist,2021-01-02 02:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10606007,Yorkville West,283.69,48,4.12,0.0,0.00,0.0,0.0,0.0,100,Clouds,overcast clouds,2023-10-31 19:00:00+00:00
10606008,Yorkville West,283.62,49,3.60,0.0,0.00,0.0,0.0,0.0,100,Clouds,overcast clouds,2023-10-31 20:00:00+00:00
10606009,Yorkville West,283.26,50,3.09,0.0,0.00,0.0,0.0,0.0,100,Clouds,overcast clouds,2023-10-31 21:00:00+00:00
10606010,Yorkville West,282.89,52,3.60,0.0,0.00,0.0,0.0,0.0,100,Clouds,overcast clouds,2023-10-31 22:00:00+00:00


In [25]:
zone_to_loc = zone_to_loc.rename(columns={'zone': 'city_name'})
zone_to_loc

,PULocationID,city_name
0,4,Alphabet City
1,12,Battery Park
2,13,Battery Park City
3,24,Bloomingdale
4,41,Central Harlem
...,...,...
62,246,West Chelsea/Hudson Yards
63,249,West Village
64,261,World Trade Center
65,262,Yorkville East


In [26]:
weather_covariate_time_features_filtered_with_id = pd.merge(weather_covariate_time_features_filtered, zone_to_loc, on='city_name', how='left')
weather_covariate_time_features_filtered_with_id

,city_name,temp,humidity,wind_speed,wind_gust,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_main,weather_description,datetime,PULocationID
0,Alphabet City,276.66,80,4.60,0.0,0.57,0.0,0.0,0.0,100,Rain,light rain,2021-01-01 23:00:00+00:00,4
1,Alphabet City,276.58,83,7.20,0.0,0.43,0.0,0.0,0.0,100,Rain,light rain,2021-01-02 00:00:00+00:00,4
2,Alphabet City,276.54,88,8.20,0.0,1.98,0.0,0.0,0.0,100,Rain,moderate rain,2021-01-02 01:00:00+00:00,4
3,Alphabet City,276.65,91,7.20,0.0,2.03,0.0,0.0,0.0,100,Rain,moderate rain,2021-01-02 02:00:00+00:00,4
4,Alphabet City,276.65,91,7.20,0.0,2.03,0.0,0.0,0.0,100,Mist,mist,2021-01-02 02:00:00+00:00,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2005552,Yorkville West,283.69,48,4.12,0.0,0.00,0.0,0.0,0.0,100,Clouds,overcast clouds,2023-10-31 19:00:00+00:00,263
2005553,Yorkville West,283.62,49,3.60,0.0,0.00,0.0,0.0,0.0,100,Clouds,overcast clouds,2023-10-31 20:00:00+00:00,263
2005554,Yorkville West,283.26,50,3.09,0.0,0.00,0.0,0.0,0.0,100,Clouds,overcast clouds,2023-10-31 21:00:00+00:00,263
2005555,Yorkville West,282.89,52,3.60,0.0,0.00,0.0,0.0,0.0,100,Clouds,overcast clouds,2023-10-31 22:00:00+00:00,263


In [28]:
unique_ids = list(weather_covariate_time_features_filtered_with_id['PULocationID'].unique())
print(len(unique_ids), "\n")
print(unique_ids)

66 

[4, 12, 13, 24, 41, 42, 43, 45, 48, 50, 68, 74, 75, 79, 87, 88, 90, 100, 107, 113, 114, 116, 120, 125, 127, 128, 137, 140, 141, 142, 143, 144, 148, 151, 152, 153, 158, 161, 162, 163, 164, 166, 170, 186, 194, 202, 209, 211, 224, 229, 230, 231, 232, 233, 234, 236, 237, 238, 239, 243, 244, 246, 249, 261, 262, 263]


In [29]:
dd_ts_fresh = pd.read_parquet("[Draft_2]_combined_dd_ss_raw_feat_data_significant_lags_pruned.parquet", engine="pyarrow")
dd_ts_fresh

,time_idx,datetime,group,target,inflow_lag_0,inflow_lag_1,inflow_lag_2,inflow_lag_3,outflow_lag_0,outflow_lag_1,...,inflow_diff2_per1__count_above_mean,inflow_diff2_per1__autocorrelation__lag_1,inflow_diff2_per1__autocorrelation__lag_2,inflow_diff2_per1__partial_autocorrelation__lag_1,inflow_diff2_per1__partial_autocorrelation__lag_2,inflow_diff2_per1__partial_autocorrelation__lag_3,"inflow_diff2_per1__fft_coefficient__attr_""real""__coeff_0",inflow_diff2_per1__autocorrelation__lag_3,inflow_diff2_per1__count_below_mean,inflow_diff2_per1__kurtosis
0,167,2021-01-07 23:00:00,4,0.0,10.0,8.0,1.0,1.0,5.0,3.0,...,0.0,-0.517121,0.028399,-0.517109,-0.362827,-0.207694,-6.0,-0.000421,0.0,0.574259
1,168,2021-01-08 00:00:00,4,1.0,4.0,13.0,0.0,2.0,0.0,1.0,...,1.0,-1.000000,0.028399,-0.517109,-0.362827,-0.207694,-3.0,-0.000421,1.0,0.574259
2,169,2021-01-08 01:00:00,4,2.0,1.0,9.0,0.0,0.0,1.0,2.0,...,2.0,-0.250000,-1.000000,-0.517109,-0.362827,-0.207694,0.0,-0.000421,1.0,0.574259
3,170,2021-01-08 02:00:00,4,0.0,1.0,5.0,0.0,1.0,2.0,3.0,...,3.0,-0.146199,-0.438596,-0.146199,-0.362827,-0.207694,2.0,-0.684211,1.0,3.750693
4,171,2021-01-08 03:00:00,4,0.0,3.0,2.0,1.0,1.0,0.0,0.0,...,3.0,-0.131773,-0.532020,-0.131773,-0.362827,-0.207694,-3.0,-0.919951,2.0,-3.126259
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631191,24808,2023-10-31 16:00:00,263,126.0,176.0,136.0,23.0,226.0,142.0,113.0,...,14.0,-0.421845,-0.103924,-0.421845,-0.342896,0.203789,-64.0,0.351349,11.0,0.235154
1631192,24809,2023-10-31 17:00:00,263,112.0,153.0,144.0,9.0,185.0,126.0,82.0,...,13.0,-0.546637,0.012524,-0.546637,-0.408289,-0.018269,10.0,0.202870,12.0,0.405906
1631193,24810,2023-10-31 18:00:00,263,99.0,135.0,94.0,7.0,136.0,112.0,77.0,...,14.0,-0.540946,0.001364,-0.540946,-0.411744,-0.030810,14.0,0.203590,11.0,0.406788
1631194,24811,2023-10-31 19:00:00,263,60.0,119.0,62.0,4.0,85.0,99.0,30.0,...,14.0,-0.443736,-0.035146,-0.443736,-0.288941,0.078741,-8.0,0.206272,11.0,0.743705


In [30]:
weather_covariate_time_features_filtered_with_id['datetime'] = weather_covariate_time_features_filtered_with_id['datetime'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
weather_covariate_time_features_filtered_with_id

,city_name,temp,humidity,wind_speed,wind_gust,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_main,weather_description,datetime,PULocationID
0,Alphabet City,276.66,80,4.60,0.0,0.57,0.0,0.0,0.0,100,Rain,light rain,2021-01-01 23:00:00,4
1,Alphabet City,276.58,83,7.20,0.0,0.43,0.0,0.0,0.0,100,Rain,light rain,2021-01-02 00:00:00,4
2,Alphabet City,276.54,88,8.20,0.0,1.98,0.0,0.0,0.0,100,Rain,moderate rain,2021-01-02 01:00:00,4
3,Alphabet City,276.65,91,7.20,0.0,2.03,0.0,0.0,0.0,100,Rain,moderate rain,2021-01-02 02:00:00,4
4,Alphabet City,276.65,91,7.20,0.0,2.03,0.0,0.0,0.0,100,Mist,mist,2021-01-02 02:00:00,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2005552,Yorkville West,283.69,48,4.12,0.0,0.00,0.0,0.0,0.0,100,Clouds,overcast clouds,2023-10-31 19:00:00,263
2005553,Yorkville West,283.62,49,3.60,0.0,0.00,0.0,0.0,0.0,100,Clouds,overcast clouds,2023-10-31 20:00:00,263
2005554,Yorkville West,283.26,50,3.09,0.0,0.00,0.0,0.0,0.0,100,Clouds,overcast clouds,2023-10-31 21:00:00,263
2005555,Yorkville West,282.89,52,3.60,0.0,0.00,0.0,0.0,0.0,100,Clouds,overcast clouds,2023-10-31 22:00:00,263


In [31]:
weather_covariate_time_features_filtered_with_id = weather_covariate_time_features_filtered_with_id.rename(columns={"PULocationID": "group"})
weather_covariate_time_features_filtered_with_id

,city_name,temp,humidity,wind_speed,wind_gust,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_main,weather_description,datetime,group
0,Alphabet City,276.66,80,4.60,0.0,0.57,0.0,0.0,0.0,100,Rain,light rain,2021-01-01 23:00:00,4
1,Alphabet City,276.58,83,7.20,0.0,0.43,0.0,0.0,0.0,100,Rain,light rain,2021-01-02 00:00:00,4
2,Alphabet City,276.54,88,8.20,0.0,1.98,0.0,0.0,0.0,100,Rain,moderate rain,2021-01-02 01:00:00,4
3,Alphabet City,276.65,91,7.20,0.0,2.03,0.0,0.0,0.0,100,Rain,moderate rain,2021-01-02 02:00:00,4
4,Alphabet City,276.65,91,7.20,0.0,2.03,0.0,0.0,0.0,100,Mist,mist,2021-01-02 02:00:00,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2005552,Yorkville West,283.69,48,4.12,0.0,0.00,0.0,0.0,0.0,100,Clouds,overcast clouds,2023-10-31 19:00:00,263
2005553,Yorkville West,283.62,49,3.60,0.0,0.00,0.0,0.0,0.0,100,Clouds,overcast clouds,2023-10-31 20:00:00,263
2005554,Yorkville West,283.26,50,3.09,0.0,0.00,0.0,0.0,0.0,100,Clouds,overcast clouds,2023-10-31 21:00:00,263
2005555,Yorkville West,282.89,52,3.60,0.0,0.00,0.0,0.0,0.0,100,Clouds,overcast clouds,2023-10-31 22:00:00,263


In [36]:
weather_covariate_time_features_filtered_with_id['datetime'] = pd.to_datetime(weather_covariate_time_features_filtered_with_id['datetime'])
weather_covariate_time_features_filtered_with_id

,city_name,temp,humidity,wind_speed,wind_gust,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_main,weather_description,datetime,group
0,Alphabet City,276.66,80,4.60,0.0,0.57,0.0,0.0,0.0,100,Rain,light rain,2021-01-01 23:00:00,4
1,Alphabet City,276.58,83,7.20,0.0,0.43,0.0,0.0,0.0,100,Rain,light rain,2021-01-02 00:00:00,4
2,Alphabet City,276.54,88,8.20,0.0,1.98,0.0,0.0,0.0,100,Rain,moderate rain,2021-01-02 01:00:00,4
3,Alphabet City,276.65,91,7.20,0.0,2.03,0.0,0.0,0.0,100,Rain,moderate rain,2021-01-02 02:00:00,4
4,Alphabet City,276.65,91,7.20,0.0,2.03,0.0,0.0,0.0,100,Mist,mist,2021-01-02 02:00:00,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2005552,Yorkville West,283.69,48,4.12,0.0,0.00,0.0,0.0,0.0,100,Clouds,overcast clouds,2023-10-31 19:00:00,263
2005553,Yorkville West,283.62,49,3.60,0.0,0.00,0.0,0.0,0.0,100,Clouds,overcast clouds,2023-10-31 20:00:00,263
2005554,Yorkville West,283.26,50,3.09,0.0,0.00,0.0,0.0,0.0,100,Clouds,overcast clouds,2023-10-31 21:00:00,263
2005555,Yorkville West,282.89,52,3.60,0.0,0.00,0.0,0.0,0.0,100,Clouds,overcast clouds,2023-10-31 22:00:00,263


In [37]:
print(list(weather_covariate_time_features_filtered_with_id['datetime'][:10]))

[Timestamp('2021-01-01 23:00:00'), Timestamp('2021-01-02 00:00:00'), Timestamp('2021-01-02 01:00:00'), Timestamp('2021-01-02 02:00:00'), Timestamp('2021-01-02 02:00:00'), Timestamp('2021-01-02 03:00:00'), Timestamp('2021-01-02 04:00:00'), Timestamp('2021-01-02 05:00:00'), Timestamp('2021-01-02 06:00:00'), Timestamp('2021-01-02 06:00:00')]


In [72]:
print(dd_ts_fresh.duplicated(['datetime', 'group']).sum())

0


In [73]:
print(weather_covariate_time_features_filtered_with_id.duplicated(['datetime', 'group']).sum())


369219


In [74]:
weather_covariate_time_features_filtered_with_id = weather_covariate_time_features_filtered_with_id.drop_duplicates(['datetime', 'group'])

In [75]:
final_df = pd.merge(dd_ts_fresh, weather_covariate_time_features_filtered_with_id, on=['datetime', 'group'], how='inner')
final_df

,time_idx,datetime,group,target,inflow_lag_0,inflow_lag_1,inflow_lag_2,inflow_lag_3,outflow_lag_0,outflow_lag_1,...,humidity,wind_speed,wind_gust,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_main,weather_description
0,167,2021-01-07 23:00:00,4,0.0,10.0,8.0,1.0,1.0,5.0,3.0,...,53,7.70,0.00,0.0,0.0,0.0,0.0,0,Clear,sky is clear
1,168,2021-01-08 00:00:00,4,1.0,4.0,13.0,0.0,2.0,0.0,1.0,...,56,8.23,9.77,0.0,0.0,0.0,0.0,0,Clear,sky is clear
2,169,2021-01-08 01:00:00,4,2.0,1.0,9.0,0.0,0.0,1.0,2.0,...,61,6.70,0.00,0.0,0.0,0.0,0.0,0,Clear,sky is clear
3,170,2021-01-08 02:00:00,4,0.0,1.0,5.0,0.0,1.0,2.0,3.0,...,63,6.20,10.30,0.0,0.0,0.0,0.0,0,Clear,sky is clear
4,171,2021-01-08 03:00:00,4,0.0,3.0,2.0,1.0,1.0,0.0,0.0,...,64,4.63,0.00,0.0,0.0,0.0,0.0,0,Clear,sky is clear
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631191,24808,2023-10-31 16:00:00,263,126.0,176.0,136.0,23.0,226.0,142.0,113.0,...,50,4.63,0.00,0.0,0.0,0.0,0.0,20,Clouds,few clouds
1631192,24809,2023-10-31 17:00:00,263,112.0,153.0,144.0,9.0,185.0,126.0,82.0,...,47,4.12,0.00,0.0,0.0,0.0,0.0,20,Clouds,few clouds
1631193,24810,2023-10-31 18:00:00,263,99.0,135.0,94.0,7.0,136.0,112.0,77.0,...,50,3.60,0.00,0.0,0.0,0.0,0.0,75,Clouds,broken clouds
1631194,24811,2023-10-31 19:00:00,263,60.0,119.0,62.0,4.0,85.0,99.0,30.0,...,48,4.12,0.00,0.0,0.0,0.0,0.0,100,Clouds,overcast clouds


In [76]:
for c in final_df.columns:
    print(c)

time_idx
datetime
group
target
inflow_lag_0
inflow_lag_1
inflow_lag_2
inflow_lag_3
outflow_lag_0
outflow_lag_1
outflow_lag_2
outflow_lag_3
inflow
inflow_diff1_per1
inflow_diff1_per2
inflow_diff2_per1
inflow_lag_0__fft_coefficient__attr_"real"__coeff_0
inflow_lag_0__fft_coefficient__attr_"real"__coeff_1
inflow_lag_0__skewness
inflow_lag_0__kurtosis
inflow_lag_0__count_above_mean
inflow_lag_0__count_below_mean
inflow_lag_0__autocorrelation__lag_1
inflow_lag_0__autocorrelation__lag_2
inflow_lag_0__autocorrelation__lag_3
inflow_lag_0__partial_autocorrelation__lag_1
inflow_lag_0__partial_autocorrelation__lag_2
inflow_lag_0__partial_autocorrelation__lag_3
inflow_lag_1__fft_coefficient__attr_"real"__coeff_0
inflow_lag_1__fft_coefficient__attr_"real"__coeff_1
inflow_lag_1__skewness
inflow_lag_1__kurtosis
inflow_lag_1__count_above_mean
inflow_lag_1__count_below_mean
inflow_lag_1__autocorrelation__lag_1
inflow_lag_1__autocorrelation__lag_2
inflow_lag_1__autocorrelation__lag_3
inflow_lag_1__parti

In [77]:
final_df.to_parquet("combined_dd_ss_weather_additional_covariate.parquet", engine="pyarrow")

In [78]:
df = pd.read_parquet("combined_dd_ss_weather_additional_covariate.parquet")
df

,time_idx,datetime,group,target,inflow_lag_0,inflow_lag_1,inflow_lag_2,inflow_lag_3,outflow_lag_0,outflow_lag_1,...,humidity,wind_speed,wind_gust,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_main,weather_description
0,167,2021-01-07 23:00:00,4,0.0,10.0,8.0,1.0,1.0,5.0,3.0,...,53,7.70,0.00,0.0,0.0,0.0,0.0,0,Clear,sky is clear
1,168,2021-01-08 00:00:00,4,1.0,4.0,13.0,0.0,2.0,0.0,1.0,...,56,8.23,9.77,0.0,0.0,0.0,0.0,0,Clear,sky is clear
2,169,2021-01-08 01:00:00,4,2.0,1.0,9.0,0.0,0.0,1.0,2.0,...,61,6.70,0.00,0.0,0.0,0.0,0.0,0,Clear,sky is clear
3,170,2021-01-08 02:00:00,4,0.0,1.0,5.0,0.0,1.0,2.0,3.0,...,63,6.20,10.30,0.0,0.0,0.0,0.0,0,Clear,sky is clear
4,171,2021-01-08 03:00:00,4,0.0,3.0,2.0,1.0,1.0,0.0,0.0,...,64,4.63,0.00,0.0,0.0,0.0,0.0,0,Clear,sky is clear
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631191,24808,2023-10-31 16:00:00,263,126.0,176.0,136.0,23.0,226.0,142.0,113.0,...,50,4.63,0.00,0.0,0.0,0.0,0.0,20,Clouds,few clouds
1631192,24809,2023-10-31 17:00:00,263,112.0,153.0,144.0,9.0,185.0,126.0,82.0,...,47,4.12,0.00,0.0,0.0,0.0,0.0,20,Clouds,few clouds
1631193,24810,2023-10-31 18:00:00,263,99.0,135.0,94.0,7.0,136.0,112.0,77.0,...,50,3.60,0.00,0.0,0.0,0.0,0.0,75,Clouds,broken clouds
1631194,24811,2023-10-31 19:00:00,263,60.0,119.0,62.0,4.0,85.0,99.0,30.0,...,48,4.12,0.00,0.0,0.0,0.0,0.0,100,Clouds,overcast clouds


In [79]:
dd_ts_fresh

,time_idx,datetime,group,target,inflow_lag_0,inflow_lag_1,inflow_lag_2,inflow_lag_3,outflow_lag_0,outflow_lag_1,...,inflow_diff2_per1__count_above_mean,inflow_diff2_per1__autocorrelation__lag_1,inflow_diff2_per1__autocorrelation__lag_2,inflow_diff2_per1__partial_autocorrelation__lag_1,inflow_diff2_per1__partial_autocorrelation__lag_2,inflow_diff2_per1__partial_autocorrelation__lag_3,"inflow_diff2_per1__fft_coefficient__attr_""real""__coeff_0",inflow_diff2_per1__autocorrelation__lag_3,inflow_diff2_per1__count_below_mean,inflow_diff2_per1__kurtosis
0,167,2021-01-07 23:00:00,4,0.0,10.0,8.0,1.0,1.0,5.0,3.0,...,0.0,-0.517121,0.028399,-0.517109,-0.362827,-0.207694,-6.0,-0.000421,0.0,0.574259
1,168,2021-01-08 00:00:00,4,1.0,4.0,13.0,0.0,2.0,0.0,1.0,...,1.0,-1.000000,0.028399,-0.517109,-0.362827,-0.207694,-3.0,-0.000421,1.0,0.574259
2,169,2021-01-08 01:00:00,4,2.0,1.0,9.0,0.0,0.0,1.0,2.0,...,2.0,-0.250000,-1.000000,-0.517109,-0.362827,-0.207694,0.0,-0.000421,1.0,0.574259
3,170,2021-01-08 02:00:00,4,0.0,1.0,5.0,0.0,1.0,2.0,3.0,...,3.0,-0.146199,-0.438596,-0.146199,-0.362827,-0.207694,2.0,-0.684211,1.0,3.750693
4,171,2021-01-08 03:00:00,4,0.0,3.0,2.0,1.0,1.0,0.0,0.0,...,3.0,-0.131773,-0.532020,-0.131773,-0.362827,-0.207694,-3.0,-0.919951,2.0,-3.126259
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631191,24808,2023-10-31 16:00:00,263,126.0,176.0,136.0,23.0,226.0,142.0,113.0,...,14.0,-0.421845,-0.103924,-0.421845,-0.342896,0.203789,-64.0,0.351349,11.0,0.235154
1631192,24809,2023-10-31 17:00:00,263,112.0,153.0,144.0,9.0,185.0,126.0,82.0,...,13.0,-0.546637,0.012524,-0.546637,-0.408289,-0.018269,10.0,0.202870,12.0,0.405906
1631193,24810,2023-10-31 18:00:00,263,99.0,135.0,94.0,7.0,136.0,112.0,77.0,...,14.0,-0.540946,0.001364,-0.540946,-0.411744,-0.030810,14.0,0.203590,11.0,0.406788
1631194,24811,2023-10-31 19:00:00,263,60.0,119.0,62.0,4.0,85.0,99.0,30.0,...,14.0,-0.443736,-0.035146,-0.443736,-0.288941,0.078741,-8.0,0.206272,11.0,0.743705
